In [26]:
histData = pd.read_excel("eq_hist_data.xlsx")
newDays = (histData['day'] - 1538438400) / (60 * 60 * 24)
histData.update({'day' : newDays})
histData.to_excel('eq_hist_data.xlsx')

In [1]:
import numpy as np
import pandas as pd
import os
os.chdir("C:/Users/dimme/Downloads/DATAsetBIOCAD")

In [2]:
import datetime

__start = '2019-03-18'
_dayOfWeek = 0

def daysBetween(_start, _end, dayOfWeek):
    l_start = _start.split('-')
    l_end= _end.split('-')
    start = datetime.date(int(l_start[0]), int(l_start[1]), int(l_start[2]))
    end = datetime.date(int(l_end[0]), int(l_end[1]), int(l_end[2]))
    diff = end - start
    days = int((str(diff)).split()[0])
    dayOfWeek -= 1
    weekends = 2
    if dayOfWeek == 6:
        weekends = 1
    days -= (((days - 7 + dayOfWeek) // 7) * 2 + 2)
    return days
        

In [3]:
#possibility function
def possibilityWork(eqId):
#eqId = equipmentData['id'][0]
    histOfEquip = histData[histData['id'] == eqId]
    timeWork = 0.0
    timeRepair = 0.0
    for day in histOfEquip.itertuples():
        if abs(day.work) > 0.001:
            timeWork += day.work
        if abs(day.maintenance) > 0.001:
            timeRepair += day.maintenance
    if (timeWork + timeRepair) < 0.001:
        return 0
    return timeWork / (timeWork + timeRepair)

#calculate all real power
def realPower(eqId, power):
    return power * possibilityWork(eqId)


realPowers = []
for equipment in equipmentData.itertuples():
        realPowers.append(realPower(equipment.id, equipment.speed_per_hour))

In [ ]:
dataRealPowers = pd.DataFrame({'realPowers': realPowers})
equipmentData = equipmentData.join(dataRealPowers)
equipmentData.to_excel('equipment.xlsx')

In [9]:
equipmentData = pd.read_excel("equipment.xlsx")
#set of all class
repeatedClasses = equipmentData['class']
classes = set(repeatedClasses)

#dict of all class with power
def equipClasses(eqData): 
    eqClassDict = {} 
    for eqClass in sorted(eqData['class'].unique()): 
        eqClassDict[eqClass] = sum(dict(tuple(eqData.groupby('class')))[eqClass]['realPowers']) 
    return eqClassDict
classesWitPower = equipClasses(equipmentData)

#list of {product id, set of classes for this product}
Equipment_class = productData['equipment_class']
prodactIds = productData['_id']
productsNeedInClasses = list(map(lambda equip, pr_id : [pr_id, set(equip[1:len(equip) - 1].replace("'", '').split(', '))], Equipment_class, prodactIds))

#deleting not use equipment
usedClasses = set()
for product in productsNeedInClasses:
    usedClasses = usedClasses.union(product[1])
classes = classes.intersection(usedClasses)

#deleting equipment, which we have not, but we want use it
for product in productsNeedInClasses:
    product[1] = classes.intersection(product[1])

#deleting not use equipment with power
haveToRemove = []
for equipment in classesWitPower:
    if not(equipment in classes):
        haveToRemove.append(equipment)
for equipment in haveToRemove:
    del classesWitPower[equipment]

productsDict = dict()
for product in productsNeedInClasses:
    productsDict[product[0]] = product[1]


In [73]:
#all about classes xlsx
classes = []
powers = []
equipts = []
products = []
inOrders = []
powDay = [[]for i in range(25)]
def PowerDay(powDayForEq, _dataOrder):
    day=1
    while day <= 25:
        dataOrder = _dataOrder[_dataOrder['days'] >= day]
        powDayForEq[day - 1].append(dataOrder['powerDay'].sum())
        day += 1

for node in classesWitPower:
    classData = equipmentData[equipmentData['class'] == node]
    countInProduct = 0
    countInOrder = 0
    for product in productsNeedInClasses: 
        if node in product[1]:
            data= orderData[orderData['product_id'] == product[0]]
            PowerDay(powDay, data)
            countInOrder += len(data)
            countInProduct += 1
    classes.append(node)
    powers.append(classesWitPower[node])
    equipts.append(len(classData))
    products.append(countInProduct)
    inOrders.append(countInOrder)
   
dataClasses = pd.DataFrame({'class': classes, 'power':powers, 'equipts': equipts, 'Inproducts':products, 'Inorders':inOrders})

i=1
while i <= 25:
    d = pd.DataFrame({str(i): powDay[i - 1]})
    dataClasses = dataClasses.join(d)
    i += 1
dataClasses.to_excel("classes.xlsx")

In [16]:
#calculate power for product 
powerForProduct = []
for product in productsNeedInClasses:
    power = 0.0
    for equipment in product[1]:
        power += classesWitPower[equipment]
    powerForProduct.append(power)
    
dataPowerForProduct = pd.DataFrame({'powerForProduct': powerForProduct})
newProductData = productData.join(dataPowerForProduct)

In [21]:
#writing expenses
productData = pd.read_excel("product.xlsx")
orderData = pd.read_excel("order.xlsx")
listOrderToEasy = []

for order in orderData.itertuples():
    order.product_id
    productFrame = productData[productData['_id'] == order.product_id]
    N = order.amount / productFrame.iloc[0].powerForProduct
    listOrderToEasy.append(N)
    
dataPowerOrder = pd.DataFrame({'expenses': listOrderToEasy})
orderData = orderData.join(dataPowerOrder)
orderData.to_excel('order.xlsx')

#calculate days to deadline
orderData = pd.read_excel("order.xlsx")
listOfDeadlines = []
for order in orderData.itertuples():
    days = daysBetween(__start, order.deadline, _dayOfWeek)
    listOfDeadlines.append(int(days))
d = pd.DataFrame({'days': listOfDeadlines})
orderData = orderData.join(d)
orderData.to_excel('order.xlsx')

In [8]:
import copy

def inputToSchedule(expenses, days, schedule):
    i = days - 1
    j = 23
    _schedule = copy.copy(schedule)
    while i >= 0:
        while j >= 0:
            t = schedule[i][j]
            if t >= expenses:
                schedule = _schedule
                schedule[i][j] =  t - expenses
                return True
            else:
                expenses -= t
                _schedule[i][j] = 0
    return False 
def CanExecute(orderByProduct, impossuble):
    schedule = np.full((25, 24), 1.0)
    for order in orderByProduct.itertuples():
        if not inputToSchedule(order.expenses, order.days, schedule):
            impossuble.append(order.Index)

orderData = pd.read_excel("order.xlsx")
productData = pd.read_excel("product.xlsx")
impossuble = []

listProductId = productData['_id']

productId = listProductId[0]
for productId in listProductId:
    orderByProduct = orderData[orderData['product_id'] == productId].sort_values(by = 'expenses')
    CanExecute(orderByProduct, impossuble)


for orderId in impossuble:
    orderData.drop(orderId)

0

In [41]:
#power a day for orders
orderData = pd.read_excel("order.xlsx")
powerDay = []

for order in orderData.itertuples():
    powerDay.append(order.expenses / order.days)
    
powerDay
d = pd.DataFrame({'powerDay': powerDay})
orderData = orderData.join(d)
orderData.to_excel('order.xlsx')



In [43]:
#sorted orderData by expenses
orderData = pd.read_excel("order.xlsx")
orderData = orderData.sort_values(by = 'expenses')
orderData.to_excel('sortedOrder.xlsx')

In [240]:
import copy
import math

def inputToSchedule(equip, day, schedule):
    i = day - 1
    j = 23
    idEq = equip[0]
    speed = equip[1]
    amount = float(equip[2])
    while i >= 0:
        while j >= 0:
            t = schedule[idEq][i][j]
            if t * speed >= amount:
                schedule[idEq][i][j] -= amount/speed
                return 0
            else:
                amount -= t * speed
                schedule[idEq][i][j] = 0
            j -= 1
        i -= 1
    return math.floor(amount) + 1


def s(order, schedule):
    day = order.days
    amount = order.amount
    setClass = productsDict[order.product_id]
    
    condList = [(dc in setClass) for dc in dataClasses['class']]
    data = dataClasses[condList].sort_values(by = str(day))
    sumOfPower = 0;
    sumOfSpeed = 0
    equipments = []#[id, speed, count product have to make]
    #searching equipment
    for classEq in data.iterrows():
        [num, item] = classEq
        sumOfPower += item[day + 4]
        sumOfSpeed += item[1]
        equipments.append([num, item[1]])
        
    #order distribution
    remain = 0.0
    sumN = 0
    for equip in equipments:
        n = amount * equip[1] / sumOfSpeed + remain
        sumN += math.floor(n)
        equip.append(math.floor(n))
        remain = n - math.floor(n)
    equipments[0][2] += (amount -sumN)
    
    _schedule = copy.copy(schedule)
    
    # filling schedule
    for equip in equipments:
        amount = inputToSchedule(equip, day, _schedule)
        if (amount != 0):
            equip[1] = 0
            remain += amount
    if remain == 0:
        schedule = _schedule
        return True
    for equip in equipments:
        if (equip[1] != 0):
            equip[2] = remain
            remain = inputToSchedule(equip, day, _schedule)
            if (remain == 0):
                schedule = _schedule
                return True
    return False

listOrderId = []
setOfEndProduct = set()
schedule = [[[1.0 for i in range(24)] for i in range(25)]for i in range(94)]
#sorted by easy to complete
def CanExecute(orderByProduct): 
    for order in orderByProduct.itertuples():
        if order.product_id in productsDict:
            if  s(order, schedule):
                listOrderId.append(order.Index)
CanExecute(sortedOrders)
len(listOrderId)

42321

In [238]:
#d = pd.DataFrame({'OrderId': listOrderId})
#d.to_excel('completeOrder.xlsx')

In [264]:
from datetime import datetime, date, time, timedelta
d = date(2019, 1, 4)
t = time(12, 30, 30 , 500000)
u = datetime.combine(d, t)

d = d + timedelta(days = 4)
print(d)
u = u + timedelta(microseconds =3600)
print(u)

2019-01-08
2019-01-04 12:30:30.503600
